In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import random

In [2]:
import random
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, random_split
from torchvision.datasets import CIFAR10

transform = transforms.Compose([
        transforms.ToTensor(),
         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
   ])
train_dataset = CIFAR10(root='./data', train=True, transform=transform, download=True)

100%|██████████| 170M/170M [00:03<00:00, 43.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [3]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(self.tanh(self.conv1(x)))
        x = self.pool(self.tanh(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # Flatten the tensor for the fully connected layer
        x = self.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
target_model = CNN()
criterion = nn.CrossEntropyLoss()
target_optimizer = optim.Adam(target_model.parameters(), lr=0.001)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

target_model = target_model.to(device)

cpu


In [ ]:
import wandb

# Initialize wandb
wandb.init(
    project="CIFAR_10_TARGET_INF",  # Replace with your project name
    config={
        "epochs": 20,
        "batch_size": train_loader.batch_size,
        "learning_rate": target_optimizer.param_groups[0]['lr'],
        "target_epsilon": 0,
        "Run": 2 # Assuming all param groups have the same LR
    }
)

In [7]:
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import datasets
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

test_dataset = CIFAR10(root='./data', train=False, transform=transform, download=True)


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified


In [ ]:
import math

In [8]:
def train(model,optimizer,loader,epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    for i, (images, labels) in enumerate(loader):

        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

            # Append predictions and labels for metric calculation
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()


    # Print the loss after each epoch
    accuracy = 100 * correct / total
    wandb.log({"train_loss": running_loss / len(loader), "train_accuracy": accuracy, "Epoch" : epoch+1})
    print(f"Epoch [{epoch+1}/{num_epochs}], Train_Loss: {running_loss / len(loader):.4f}, Train_Accuracy: {accuracy:.4f}")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
def test(model,test_loader, epoch):
    model.eval()
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            # Append predictions and labels for metric calculation
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate metrics
    accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_predictions, average='macro')
    recall = recall_score(all_labels, all_predictions, average='macro')
    f1 = f1_score(all_labels, all_predictions, average='macro')

    #wandb.log({"test_accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1, "Epoch" : epoch+1})

    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")


In [ ]:
def fit(model,optimizer,train_loader,test_loader):

    for epoch in range(num_epochs):
      train(model,optimizer,train_loader,epoch)
      test(model,test_loader,epoch)

In [ ]:
num_epochs = 20

In [ ]:
fit(target_model,target_optimizer,train_loader,test_loader)

In [ ]:
torch.save(target_model.state_dict(), 'target_model_eps_inf_run2.pt')

In [ ]:
test(target_model,train_loader, 1)

In [10]:
target_model.load_state_dict(torch.load('target_model_eps_inf.pt', map_location=torch.device('cpu')))

<ipython-input-10-0672307f7cfd>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  target_model.load_state_dict(torch.load('target_model_eps_inf.pt', map_location=torch.device

<All keys matched successfully>

In [19]:
softmax = nn.Softmax(dim=1)

In [20]:
def generate_attack_data(model, data_loader, member_label):
    model.eval()
    attack_data = []
    attack_labels = []
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            probs = softmax(model(images))
            attack_data.append(probs.cpu().numpy())
            attack_labels.append(member_label)
    return attack_data, attack_labels

In [21]:
train_loader_resized = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)
test_loader_resized = DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)

In [22]:
attack_data = []
attack_labels = []
member_data, member_labels = generate_attack_data(target_model, train_loader_resized, member_label=1)
non_member_data, non_member_labels = generate_attack_data(target_model, test_loader_resized, member_label=0)
attack_data.extend(member_data+non_member_data)
attack_labels.extend(member_labels+non_member_labels)

In [33]:
print(sum(attack_data[3]))

0.999999905639454


In [26]:
import numpy as np
attack_data = np.array(attack_data).reshape(len(attack_data), -1)
y_test = np.array(attack_labels)
np.save("attack_data.npy",attack_data)
np.save("membership_labels.npy",y_test)